In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

In [2]:
data=pd.read_csv('O2O数据.csv',
           parse_dates=['Date_received','Date'])

In [3]:
# 先假定所有用户都是普通用户
## 没有领优惠券，过来消费： 普通用户（-1）
data['label']=-1
## 领了优惠券，15天以内过来消费：正（1）
true_index=(data['Date']-data['Date_received']).dt.days<=15
data.loc[true_index,'label']=1
## 领了优惠券，领了优惠券没消费：负（0）
False_index2=(data['Date_received'].notnull()&data['Date'].isnull())
## 领了优惠券，过了15天后来消费：负（0）
False_index1=(data['Date']-data['Date_received']).dt.days>15
data.loc[(False_index1|False_index2),'label']=0

In [4]:
# 用户距离存在空值
data['Distance'].isnull().sum()
# 对空值进行填充
#例1：直接对控制进行赋值
data.loc[data['Distance'].isnull(),'Distance']=data['Distance'].mean()
#例2：利用函数填充一个数
data['Distance'].fillna(value=data['Distance'].mean())
#例3：利用相邻数据进行填充
data['Distance'].fillna(method='ffill',  # bfill
                       inplace=False)

0          0.0
1          1.0
2          0.0
3          0.0
4          0.0
          ... 
1048570    1.0
1048571    1.0
1048572    1.0
1048573    1.0
1048574    1.0
Name: Distance, Length: 1048575, dtype: float64

In [5]:
def fun(i):
    if ':' in str(i):
        list1=str(i).split(':')
        return 1-int(list1[1])/int(list1[0])
    elif '.' in str(i):
        return float(i)
    else:
        return 1
data['Discount_rate_new']=data['Discount_rate'].apply(fun)

In [6]:
coupon_popu=data[['Coupon_id','label']].groupby('Coupon_id').agg(lambda x: sum(x==1)/len(x))
coupon_popu.columns=['coupon_popu']
# 把求的结果拼回去
data=pd.merge(data,coupon_popu,
             left_on='Coupon_id',right_index=True,
             how='left')

In [7]:
user_get=data[['User_id','Coupon_id']].groupby('User_id').agg(lambda x: sum(x.notnull()))
user_get.columns=['user_get']
data=pd.merge(data,user_get,
             left_on='User_id',right_index=True,
             how='left')

### 作业1：统计各用户 label 为1的数量

In [8]:
label_1=data[['User_id','label']].groupby('User_id').agg(lambda x: sum(x==1))
label_1.columns=['label_1']
data=pd.merge(data,label_1,
             left_on='User_id',right_index=True,
             how='left')
label_1

,label_1
User_id,
4,0
35,0
64,0
144,0
147,0
...,...
7360928,1
7360952,0
7360961,0


In [9]:
merchat_popu=data[['Merchant_id','Coupon_id','label']].groupby('Merchant_id').agg({
                                            'label':lambda x:sum(x==1),
                                            'Coupon_id':lambda x: sum(x.notnull())})
merchat_popu['merchat_popu']=merchat_popu['label']/merchat_popu['Coupon_id']
data=pd.merge(data,merchat_popu[['merchat_popu']],
             left_on='Merchant_id',right_index=True,
             how='left')

In [10]:
UM_used=data[['User_id','Merchant_id','label']].groupby(['User_id','Merchant_id']).agg(
                                                            lambda x: sum(x==1))
UM_used.columns=['UM_used']
data=pd.merge(data,UM_used,
             left_on=['User_id','Merchant_id'],
            right_index=True,
             how='left')

### 作业2：用户在商家领取的优惠券数

In [11]:
data

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,Discount_rate_new,coupon_popu,user_get,label_1,merchat_popu,UM_used
0,1439408,2632,NaN,NaN,0.0,NaT,2016-02-17,-1,1.000000,NaN,5,0,0.000000,0
1,1439408,4663,11002.0,150:20:00,1.0,2016-05-28,NaT,0,0.866667,0.001727,5,0,0.004378,0
2,1439408,2632,8591.0,20:01,0.0,2016-02-17,NaT,0,0.950000,0.000000,5,0,0.000000,0
3,1439408,2632,1078.0,20:01,0.0,2016-03-19,NaT,0,0.950000,0.000000,5,0,0.000000,0
4,1439408,2632,8591.0,20:01,0.0,2016-06-13,NaT,0,0.950000,0.000000,5,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,5186127,4043,3996.0,30:05:00,1.0,2016-02-03,2016-02-10,1,0.833333,0.222857,2,1,0.045272,1
1048571,5186127,4043,3996.0,30:05:00,1.0,2016-02-12,NaT,0,0.833333,0.222857,2,1,0.045272,1
1048572,5186127,4043,NaN,NaN,1.0,NaT,2016-05-30,-1,1.000000,NaN,2,1,0.045272,1
1048573,5186127,4043,NaN,NaN,1.0,NaT,2016-05-01,-1,1.000000,NaN,2,1,0.045272,1


In [12]:
user_M_get=data[['User_id','Merchant_id','label']].groupby(['User_id','Merchant_id']).count()
user_M_get

label
User_id Merchant_id       
4       1433             1
        1469             1
35      3381             4
64      2146             1
144     1553             1
...                    ...
7360928 3532             7
7360952 3403             1
7360961 3621             1
7360966 3381             1
7361024 1433             1

[550972 rows x 1 columns]

In [13]:
user_M_get.columns=['user_M_get']
data=pd.merge(data,user_M_get,
             left_on=['User_id','Merchant_id'],right_index=True,
             how='left')

### 作业3：用户在商家消费的次数

In [14]:
data['labelX']=0
## 普通消费日期
False_index2X=(data['Date'].notnull()&data['Coupon_id'].isnull())
## 用优惠券消费日期
False_index1X=(data['Date'].notnull()&data['Coupon_id'].notnull())
data.loc[(False_index1X|False_index2X),'labelX']=1

In [15]:
user_M_C=data[['User_id','Merchant_id','labelX']].groupby(['User_id','Merchant_id']).agg(lambda x: sum(x==1))
user_M_C

labelX
User_id Merchant_id        
4       1433              0
        1469              0
35      3381              0
64      2146              0
144     1553              0
...                     ...
7360928 3532              6
7360952 3403              0
7360961 3621              0
7360966 3381              0
7361024 1433              0

[550972 rows x 1 columns]

In [16]:
user_M_C.columns=['user_M_C']
data=pd.merge(data,user_M_C,
             left_on=['User_id','Merchant_id'],right_index=True,
             how='left')

In [17]:
data

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,Discount_rate_new,coupon_popu,user_get,label_1,merchat_popu,UM_used,user_M_get,labelX,user_M_C
0,1439408,2632,NaN,NaN,0.0,NaT,2016-02-17,-1,1.000000,NaN,5,0,0.000000,0,6,1,3
1,1439408,4663,11002.0,150:20:00,1.0,2016-05-28,NaT,0,0.866667,0.001727,5,0,0.004378,0,1,0,0
2,1439408,2632,8591.0,20:01,0.0,2016-02-17,NaT,0,0.950000,0.000000,5,0,0.000000,0,6,0,3
3,1439408,2632,1078.0,20:01,0.0,2016-03-19,NaT,0,0.950000,0.000000,5,0,0.000000,0,6,0,3
4,1439408,2632,8591.0,20:01,0.0,2016-06-13,NaT,0,0.950000,0.000000,5,0,0.000000,0,6,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,5186127,4043,3996.0,30:05:00,1.0,2016-02-03,2016-02-10,1,0.833333,0.222857,2,1,0.045272,1,6,1,5
1048571,5186127,4043,3996.0,30:05:00,1.0,2016-02-12,NaT,0,0.833333,0.222857,2,1,0.045272,1,6,0,5
1048572,5186127,4043,NaN,NaN,1.0,NaT,2016-05-30,-1,1.000000,NaN,2,1,0.045272,1,6,1,5
1048573,5186127,4043,NaN,NaN,1.0,NaT,2016-05-01,-1,1.000000,NaN,2,1,0.045272,1,6,1,5


In [18]:
data['label'].value_counts()
n=10
data_T=data[data['label']==1].sample(n)

### 抽样负样本

In [19]:
data_F=data[data['label']==-1].sample(n)
data_F

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,Discount_rate_new,coupon_popu,user_get,label_1,merchat_popu,UM_used,user_M_get,labelX,user_M_C
931346,5966578,685,NaN,NaN,0.0,NaT,2016-06-07,-1,1.0,NaN,17,5,0.395833,5,51,1,40
591583,3080046,5212,NaN,NaN,0.0,NaT,2016-06-25,-1,1.0,NaN,0,0,0.666667,0,3,1,3
278915,7238597,3381,NaN,NaN,0.0,NaT,2016-01-23,-1,1.0,NaN,6,1,0.011912,1,6,1,6
420,2904416,1080,NaN,NaN,0.0,NaT,2016-05-23,-1,1.0,NaN,4,0,0.016634,0,5,1,3
154175,4116062,4885,NaN,NaN,10.0,NaT,2016-03-28,-1,1.0,NaN,2,0,0.368932,0,9,1,7
852664,4875037,4142,NaN,NaN,0.0,NaT,2016-03-28,-1,1.0,NaN,8,6,0.317468,6,73,1,73
997828,3147361,6554,NaN,NaN,0.0,NaT,2016-05-23,-1,1.0,NaN,2,0,0.000000,0,4,1,2
596211,1473979,5557,NaN,NaN,6.0,NaT,2016-04-10,-1,1.0,NaN,1,0,0.118421,0,2,1,1
483099,6456175,8497,NaN,NaN,10.0,NaT,2016-05-21,-1,1.0,NaN,3,0,0.027778,0,2,1,1
794246,2941626,4453,NaN,NaN,10.0,NaT,2016-04-17,-1,1.0,NaN,1,0,0.000000,0,27,1,26


### 合并抽样的正负样本

In [20]:
data_concat=pd.concat([data_T,data_F])
data_concat

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,Discount_rate_new,coupon_popu,user_get,label_1,merchat_popu,UM_used,user_M_get,labelX,user_M_C
334318,4946582,6901,11909.0,0.95,0.000000,2016-05-28,2016-05-28,1,0.950000,0.089431,12,1,0.058036,1,4,1,3
400388,2751537,7910,11637.0,30:05:00,0.000000,2016-05-12,2016-05-12,1,0.833333,0.550459,119,118,0.405240,118,154,1,154
586962,4307660,7838,12429.0,0.95,2.000000,2016-05-21,2016-05-21,1,0.950000,0.114286,3,1,0.063147,1,2,1,1
316119,3810057,797,12331.0,30:05:00,0.000000,2016-05-08,2016-05-13,1,0.833333,0.243513,10,4,0.232472,4,11,1,7
964759,4086253,7422,4727.0,0.8,5.000000,2016-05-01,2016-05-01,1,0.800000,0.205207,1,1,0.193798,1,1,1,1
696995,881287,3381,1807.0,300:30:00,1.000000,2016-02-04,2016-02-04,1,0.900000,0.004864,4,1,0.011912,1,2,1,2
412886,3637341,3284,4481.0,30:01:00,0.000000,2016-01-20,2016-01-29,1,0.966667,0.122862,4,1,0.228889,1,6,1,5
5634,6235860,2934,5686.0,30:05:00,1.000000,2016-03-23,2016-03-27,1,0.833333,0.157312,2,1,0.157312,1,3,1,3
766658,1826119,6485,10323.0,20:05,0.000000,2016-03-27,2016-03-28,1,0.750000,0.299484,4,4,0.233439,2,4,1,4
698206,6318140,1520,1468.0,50:05:00,2.362526,2016-01-17,2016-01-21,1,0.900000,0.099115,3,1,0.128457,1,1,1,1


### 预测

In [115]:
X1=data_T[:9]
Y1=data_T[9:10]

In [116]:
X1

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,Discount_rate_new,coupon_popu,user_get,label_1,merchat_popu,UM_used,user_M_get,labelX,user_M_C
334318,4946582,6901,11909.0,0.95,0.0,2016-05-28,2016-05-28,1,0.950000,0.089431,12,1,0.058036,1,4,1,3
400388,2751537,7910,11637.0,30:05:00,0.0,2016-05-12,2016-05-12,1,0.833333,0.550459,119,118,0.405240,118,154,1,154
586962,4307660,7838,12429.0,0.95,2.0,2016-05-21,2016-05-21,1,0.950000,0.114286,3,1,0.063147,1,2,1,1
316119,3810057,797,12331.0,30:05:00,0.0,2016-05-08,2016-05-13,1,0.833333,0.243513,10,4,0.232472,4,11,1,7
964759,4086253,7422,4727.0,0.8,5.0,2016-05-01,2016-05-01,1,0.800000,0.205207,1,1,0.193798,1,1,1,1
696995,881287,3381,1807.0,300:30:00,1.0,2016-02-04,2016-02-04,1,0.900000,0.004864,4,1,0.011912,1,2,1,2
412886,3637341,3284,4481.0,30:01:00,0.0,2016-01-20,2016-01-29,1,0.966667,0.122862,4,1,0.228889,1,6,1,5
5634,6235860,2934,5686.0,30:05:00,1.0,2016-03-23,2016-03-27,1,0.833333,0.157312,2,1,0.157312,1,3,1,3
766658,1826119,6485,10323.0,20:05,0.0,2016-03-27,2016-03-28,1,0.750000,0.299484,4,4,0.233439,2,4,1,4


In [117]:
X=X1.drop(labels='Discount_rate',axis=1)
Y1=Y1.drop(labels='Discount_rate',axis=1)
X['Date_received']=(X['Date_received']-X['Date_received'].min()).dt.days
X['Date']=(X['Date']-X['Date'].min()).dt.days

In [118]:
Y=X['label'].values
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [119]:
cols=['Discount_rate_new','Distance','Date_received','Date','coupon_popu','user_get','label_1','merchat_popu','UM_used','user_M_get','user_M_C']
X=X[cols].values
Y1['Date_received']=(Y1['Date_received']-Y1['Date_received'].min()).dt.days
Y1['Date']=(Y1['Date']-Y1['Date'].min()).dt.days
Y1=np.array(Y1)

In [127]:
from sklearn.decomposition import PCA
Pmodel=PCA(9)
X=Pmodel.fit_transform(X)

In [128]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_Y,test_Y=train_test_split(X,Y)

In [129]:
from sklearn.tree import DecisionTreeRegressor
Dmodel=DecisionTreeRegressor().fit(train_X,train_Y)
Pre_Y=Dmodel.predict(test_X)
Pre_Y

array([1., 1., 1.])

In [130]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_Y,Pre_Y)

0.0